1.2. Extracción de Detalles de Películas con Selenium
Una vez que tengan una lista de películas de la API, el siguiente paso es obtener información más detallada sobre ellas. Utilizarás la herramienta de automatización web Selenium para navegar a sitios web de reseñas de películas, en este caso IMDB, y extraer detalles como calificaciones, actores y directores.

Necesitamos la siguiente información:

Puntuación de IMDB (en caso de que la tenga).

Dirección (director/a o directores/as de cada película).

Guionistas (de cada película).

Argumento.

Nombre de la película.

Id de la película.

Bonus: También puedes extraer la siguiente información:

Duración de la película.

Puntuación de Rotten Tomatoes.

In [1]:
# Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) 

PASOS

1. Generar driver
2. Llamamos a una funcion que recibe un navegador y un identificador de película y devuelve los datos de esa pelicula
3. Hacemos un bucle que recorra los identificadores de las peliculas que queremos explorar. 


In [2]:
# 1 GENERAR DRIVER

def generar_driver ():

    #Inicializamos el navegador.
    driver = webdriver.Chrome()

    sleep(2)

    #Maximizamos pantalla.
    driver.maximize_window()
    print("Maximizando la web.")

    sleep(2)


    #Navegamos a la página web "https://www.imdb.com/".
    driver.get("https://www.imdb.com/")
    print("Accediendo a la web.")
 

    sleep(2)

    #Aceptamos cookies.
    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
    print("Cookies aceptadas.")

    sleep(2)

    return driver


# mi_driver = generar_driver()
# type(mi_driver)



En lugar de pasarle el nombre de la película por un input, se lo pasamos por parámetro a la funcion.

Además, le vamos a pasar el driver por parámetro para que no tenga que estar entrando desde cero a IMDB con cada película.
👇

In [3]:
import re
from time import sleep
 
# Función para extraer los datos de una película usando su ID de IMDb
def extraer_datos_pelicula(driver, id_pelicula):
    # Construimos la URL de la película en IMDb usando su ID
    url_pelicula = f"https://www.imdb.com/title/{id_pelicula}/"
    driver.get(url_pelicula)
    print(f"Accediendo a la película con ID {id_pelicula}")

    sleep(2)

    #Ahora estamos dentro de la info de la película. Vamos a sacar los datos que se nos piden:

    #Nombre de la película.
    titulo_pelicula = driver.find_element("css selector", "h1").text
    print("Nombre de la película:", titulo_pelicula)

    #Puntuación de IMDb.
    try:
        puntuacion_pelicula = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-3.kNMkxz > div.sc-3a4309f8-0.jJkxPn.sc-1f50b7c-1.doVJqQ > div > div:nth-child(1) > a > span > div > div.sc-eb51e184-0.ghvwpw > div.sc-eb51e184-2.fAhXAe > span.sc-eb51e184-1.ljxVSS").text
    except:
        puntuacion_pelicula = "No disponible"
        print("Puntuación de IMDb:", puntuacion_pelicula)

    #Info dirección.
    try:
        directores = driver.find_elements("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-4.ILcwq > div.sc-491663c0-6.bvzCJs > div.sc-491663c0-10.tkbDg > section > div.sc-1f50b7c-3.gLpgJQ > div > ul > li:nth-child(1) > div")
        nombres_directores = [director.text.strip() for director in directores]
        dato_direccion = ", ".join(nombres_directores)
        print("Dirección:", dato_direccion)
    except:
        dato_direccion = "No disponible."
        print(dato_direccion)

    #Info guionistas.
    try:
        guionistas = driver.find_elements("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-4.ILcwq > div.sc-491663c0-6.bvzCJs > div.sc-491663c0-10.tkbDg > section > div.sc-1f50b7c-3.gLpgJQ > div > ul > li:nth-child(2) > div")
        nombre_guionistas = [guionista.text.strip() for guionista in guionistas]
        
        # Usamos una expresión regular que detecte nombres completos (Nombre Apellido)
        def separar_nombres_completos(texto):
            # Patrón que busca pares de palabras con la estructura de un nombre completo
            return re.sub(r'([A-Z][a-z]+ [A-Z][a-z]+)', r'\1, ', texto)  # Añade coma y espacio
        
        # Aplicamos la función a los guionistas
        nombre_guionistas = [separar_nombres_completos(guionista) for guionista in nombre_guionistas]
        
        # Unimos los guionistas correctamente separados por comas y espacios
        dato_guion = " ".join(nombre_guionistas).rstrip(', ')  # Eliminamos la última coma y espacio, si queda
        print("Guionistas:", dato_guion)
    except:
        dato_guion = "No disponible."
        print(dato_guion)

    #Info argumento.
    try:
        argumento_pelicula = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-4.ILcwq > div.sc-491663c0-6.bvzCJs > div.sc-491663c0-10.tkbDg > section > p > span.sc-2d37a7c7-2.ggeRnl").text
    except:
        argumento_pelicula = "No disponible"
    print("Argumento:", argumento_pelicula)

    # ID de la película (sacarlo de la URL)
    id_pelicula = driver.current_url.split('/')[4]
    print("ID de la película:", id_pelicula)

    sleep(2)

    # Devolvemos los resultados que hemos obtenido en una tupla
    tupla_datos_pelicula = (titulo_pelicula, puntuacion_pelicula, dato_direccion, dato_guion, argumento_pelicula, id_pelicula)
    
    # Imprimir la tupla de resultados
    print("Datos extraídos:", tupla_datos_pelicula)
    
    return tupla_datos_pelicula

'tt0102362', 'tt0113092', 'tt0113086', 'tt0116748', 'tt0116391', 'tt0115937', 
    'tt0118694', 'tt0119495', 'tt0119806', 'tt0120202', 'tt0123003', 'tt0120263', 'tt0123055', 
    'tt0120630', 'tt0116628', 'tt0119273', 'tt0120755', 'tt0120913', 'tt0120673', 'tt0120903', 
    'tt0120917', 'tt0130623', 'tt0132910', 'tt0131704', 'tt0138749', 'tt0144964', 'tt0144574', 'tt0146984', 
    'tt0128158', 'tt0134084', 'tt0134983', 'tt0134847', 'tt0137115', 'tt0144084', 'tt0148120', 'tt0156020',
    'tt0120753', 'tt0127349', 'tt0160495', 'tt0160644', 'tt0160289', 'tt0161081', 'tt0162711', 'tt0113026', 
    'tt0119866', 'tt0122459', 'tt0119966', 'tt0120467', 'tt0122161', 'tt0120117', 'tt0120698'

In [9]:
def extractor_idowners (archivo_csv) :
    # Esta función recibe un archivo .csv y devuelve una lista con todos los idowners que hay en el archivo.
    df_pelis_API = pd.read_csv(archivo_csv) 
    lista_idowners = df_pelis_API['IdOwner'].tolist()
    return lista_idowners

In [8]:
#En un inicio raneamos para probar con varios ids:
ids_peliculas = ['tt0102362', 'tt0113092', 'tt0113086', 'tt0116748', 'tt0116391', 'tt0115937', 
    'tt0118694', 'tt0119495', 'tt0119806', 'tt0120202', 'tt0123003', 'tt0120263', 'tt0123055', 
    'tt0120630', 'tt0116628', 'tt0119273', 'tt0120755', 'tt0120913', 'tt0120673', 'tt0120903', 
    'tt0120917', 'tt0130623', 'tt0132910', 'tt0131704']
    
mi_driver = generar_driver()

#Lista para almacenar los datos
datos_peliculas = []

for ids in ids_peliculas:
    mis_datos = extraer_datos_pelicula(mi_driver, ids)
    datos_peliculas.append(mis_datos)

mi_driver.quit()

# Crear un DataFrame con los datos extraídos
columnas = ["Título", "Puntuación IMDb", "Directores", "Guionistas", "Argumento", "ID"]
df_peliculas = pd.DataFrame(datos_peliculas, columns=columnas)

# Mostrar el DataFrame
print(df_peliculas)

Maximizando la web.
Accediendo a la web.
Cookies aceptadas.
Accediendo a la película con ID tt0102362
Nombre de la película: Istota
Dirección: 
Guionistas: 
Argumento: No disponible
ID de la película: tt0102362
Datos extraídos: ('Istota', '6,3', '', '', 'No disponible', 'tt0102362')
Accediendo a la película con ID tt0113092
Nombre de la película: Por la causa
Dirección: 
Guionistas: 
Argumento: No disponible
ID de la película: tt0113092
Datos extraídos: ('Por la causa', '3,4', '', '', 'No disponible', 'tt0113092')
Accediendo a la película con ID tt0113086
Nombre de la película: Florentino y el diablo
Dirección: 
Guionistas: 
Argumento: No disponible
ID de la película: tt0113086
Datos extraídos: ('Florentino y el diablo', '7,0', '', '', 'No disponible', 'tt0113086')
Accediendo a la película con ID tt0116748
Nombre de la película: Karobaar: The Business of Love
Dirección: 
Guionistas: 
Argumento: No disponible
ID de la película: tt0116748
Datos extraídos: ('Karobaar: The Business of Love

In [11]:
#Y separados Dataframe:
df_peliculas.head()

,Título,Puntuación IMDb,Directores,Guionistas,Argumento,ID
0,Istota,"6,3",,,No disponible,tt0102362
1,Por la causa,"3,4",,,No disponible,tt0113092
2,Florentino y el diablo,"7,0",,,No disponible,tt0113086
3,Karobaar: The Business of Love,"3,7",,,No disponible,tt0116748
4,Gang,"6,3",,,No disponible,tt0116391


In [12]:
import pandas as pd

df_peliculas.to_csv('extraccion_peliculas_selenium.csv')
print("Archivo 'extraccion_peliculas_selenium.csv'' creado exitosamente.")

Archivo 'extraccion_peliculas_selenium.csv'' creado exitosamente.


def webscrapping_imbd

In [13]:
#Peliculas de 1995 a 1999:

ids_peliculas = extractor_idowners('peliculas_1995_1999.csv')

# Crear el driver de Selenium
mi_driver = generar_driver()

# Lista para almacenar los datos extraídos
datos_peliculas = []

# Bucle para recorrer todos los IDs Owners y extraer la información
for id_owner in ids_peliculas:
    mis_datos = extraer_datos_pelicula(mi_driver, id_owner)
    datos_peliculas.append(mis_datos)

# Cerrar el driver después de la extracción
mi_driver.quit()

# Convertir los datos extraídos a un DataFrame
df_peliculas = pd.DataFrame(datos_peliculas, columns=["Título", "Puntuación IMDb", "Directores", "Guionistas", "Argumento", "ID"])

Maximizando la web.
Accediendo a la web.
Cookies aceptadas.
Accediendo a la película con ID tt0100267
Nombre de la película: Nobat e Asheghi
Dirección: 
Guionistas: 
Argumento: No disponible
ID de la película: tt0100267
Datos extraídos: ('Nobat e Asheghi', '6,4', '', '', 'No disponible', 'tt0100267')
Accediendo a la película con ID tt0102739
Nombre de la película: ¡Que vivan los crotos!
Dirección: 
Guionistas: 
Argumento: No disponible
ID de la película: tt0102739
Datos extraídos: ('¡Que vivan los crotos!', '6,7', '', '', 'No disponible', 'tt0102739')
Accediendo a la película con ID tt0105096
Nombre de la película: Paraíso Perdido
Dirección: 
Guionistas: 
Argumento: No disponible
ID de la película: tt0105096
Datos extraídos: ('Paraíso Perdido', '6,4', '', '', 'No disponible', 'tt0105096')
Accediendo a la película con ID tt0105298
Nombre de la película: Nacidos para la libertad
Dirección: Dee McLachlan
Guionistas: John Varty, Andrea Buck, Dee Mc, Lachlan
Argumento: A filmmaker determine

In [17]:
# Guardar el DataFrame en un nuevo archivo CSV
df_peliculas.to_csv('peliculas_ext_sel_1995_1999.csv', index=False)
print("Archivo 'peliculas_ext_sel_1995_1999.csv' creado exitosamente.")

Archivo 'peliculas_ext_sel_1995_1999.csv' creado exitosamente.


In [24]:
#Peliculas de 2000 a 2004:

ids_peliculas = extractor_idowners('peliculas_2000_2004.csv')

# Crear el driver de Selenium
mi_driver = generar_driver()

# Lista para almacenar los datos extraídos
datos_peliculas = []

# Bucle para recorrer todos los IDs Owners y extraer la información
for id_owner in ids_peliculas:
    mis_datos = extraer_datos_pelicula(mi_driver, id_owner)
    datos_peliculas.append(mis_datos)

# Cerrar el driver después de la extracción
mi_driver.quit()

# Convertir los datos extraídos a un DataFrame
df_peliculas = pd.DataFrame(datos_peliculas, columns=["Título", "Puntuación IMDb", "Directores", "Guionistas", "Argumento", "ID"])

Maximizando la web.
Accediendo a la web.
Cookies aceptadas.
Accediendo a la película con ID tt0102362
Nombre de la película: Istota
Dirección: 
Guionistas: 
Argumento: No disponible
ID de la película: tt0102362
Datos extraídos: ('Istota', '6,3', '', '', 'No disponible', 'tt0102362')
Accediendo a la película con ID tt0113092
Nombre de la película: Por la causa
Dirección: 
Guionistas: 
Argumento: No disponible
ID de la película: tt0113092
Datos extraídos: ('Por la causa', '3,4', '', '', 'No disponible', 'tt0113092')
Accediendo a la película con ID tt0113086
Nombre de la película: Florentino y el diablo
Dirección: 
Guionistas: 
Argumento: No disponible
ID de la película: tt0113086
Datos extraídos: ('Florentino y el diablo', '7,0', '', '', 'No disponible', 'tt0113086')
Accediendo a la película con ID tt0116748
Nombre de la película: Karobaar: The Business of Love
Dirección: 
Guionistas: 
Argumento: No disponible
ID de la película: tt0116748
Datos extraídos: ('Karobaar: The Business of Love

In [25]:
# Guardar el DataFrame en un nuevo archivo CSV
df_peliculas.to_csv('peliculas_ext_sel_2000_2004.csv', index=False)
print("Archivo 'peliculas_ext_sel_2000_2004.csv' creado exitosamente.")

Archivo 'peliculas_ext_sel_2000_2004.csv' creado exitosamente.


In [21]:
#Peliculas de 2005 a 2009:

ids_peliculas = extractor_idowners('peliculas_2005_2009.csv')

# Crear el driver de Selenium
mi_driver = generar_driver()

# Lista para almacenar los datos extraídos
datos_peliculas = []

# Bucle para recorrer todos los IDs Owners y extraer la información
for id_owner in ids_peliculas:
    mis_datos = extraer_datos_pelicula(mi_driver, id_owner)
    datos_peliculas.append(mis_datos)

# Cerrar el driver después de la extracción
mi_driver.quit()

# Convertir los datos extraídos a un DataFrame
df_peliculas = pd.DataFrame(datos_peliculas, columns=["Título", "Puntuación IMDb", "Directores", "Guionistas", "Argumento", "ID"])

Maximizando la web.
Accediendo a la web.
Cookies aceptadas.
Accediendo a la película con ID tt0118141
Nombre de la película: What Is It?
Dirección: 
Guionistas: 
Argumento: No disponible
ID de la película: tt0118141
Datos extraídos: ('What Is It?', '5,3', '', '', 'No disponible', 'tt0118141')
Accediendo a la película con ID tt0121164
Nombre de la película: La novia cadáver
Dirección: Tim BurtonMike Johnson
Guionistas: Tim Burton, Carlos Grangel, John August
Argumento: Cuando un tímido novio practica sus votos matrimoniales ante la presencia inadvertida de una joven fallecida, esta se levanta de la tumba asumiendo que la ha tomado por esposa.
ID de la película: tt0121164
Datos extraídos: ('La novia cadáver', '7,4', 'Tim BurtonMike Johnson', 'Tim Burton, Carlos Grangel, John August', 'Cuando un tímido novio practica sus votos matrimoniales ante la presencia inadvertida de una joven fallecida, esta se levanta de la tumba asumiendo que la ha tomado por esposa.', 'tt0121164')
Accediendo a l

In [ ]:
# Guardar el DataFrame en un nuevo archivo CSV
df_peliculas.to_csv('peliculas_ext_sel_2005_2009.csv', index=False)
print("Archivo 'peliculas_ext_sel_2005_2009.csv' creado exitosamente.")

Archivo 'peliculas_ext_sel_2005_2009.csv' creado exitosamente.
